In [1]:
from BinanceBackTest import *
import asyncio
import nest_asyncio
import os
import utils
from pprint import pprint
import Analysis
import DataProcess
from collections import defaultdict
from datetime import datetime

nest_asyncio.apply()

print(f"작업시작 >> {datetime.now()}")

# symbols 지정
symbols = [
    "BTCUSDT",
    "XRPUSDT",
    "ADAUSDT",
    # "NOTUSDT",
    # "SANDUSDT",
    # "ARKMUSDT",
    # "SOLUSDT",
    "DOGEUSDT",
]
# interval 지정.
# intervals = ["1m", "5m", "1h"]
intervals = ["1m", "3m", "5m", "1h"]

# 적용할 데이터의 기간 지정.
start_date = "2024-8-1"
end_date = "2024-8-7"


print(f"instance 로딩 완료 >> {datetime.now()}")
obj_process = DataProcess.TradeStopper()
obj_analy = Analysis.AnalysisManager(back_test=True)
obj_analy.intervals = intervals
obj_order = OrderManager()
obj_con = DataProcess.OrderConstraint()
obj_data = DataManager(symbols=symbols,
                       intervals=intervals,
                       start_date=start_date,
                       end_date=end_date)

path = os.path.join(os.path.dirname(os.getcwd()), 'DataStore/closing_sync_data.pkl')
# data = utils._load_json(file_path=path)
# kline_data = asyncio.run(obj_data.generate_kline_interval_data(save=True))

# kline_data = utils._convert_to_array(kline_data=kline_data)
# kline_data = obj_data.generate_kline_closing_sync(kline_data=kline_data, save=True)
# symbol_map, interval_map, data_c = utils._convert_to_container(kline_data)
# obj_data.get_indices_data(data_container=data_c, lookback_days=1)

with open(path, 'rb')as file:
    data = pickle.load(file)

obj_analysis = Analysis.AnalysisManager()



trade_data = []

#손익비용 저장할 변수
pnl_data = defaultdict(float)
pnl_count = defaultdict(int)

#초기 투자 자본
seed_money = 1000

# 가장 지갑 생성
obj_wallet = WalletManager(initial_fund=seed_money)

obj_con.calc_fund(1000)

작업시작 >> 2024-12-09 21:21:28.173591
instance 로딩 완료 >> 2024-12-09 21:21:28.177460


{'count': 8,
 'targetValue': 640,
 'safetyValue': 224.0,
 'usableValue': 416.0,
 'tradeValue': 52.0}

In [2]:

data = data['BTCUSDT']['5m']
obj_analysis.case_1_data_length(kline_data_lv3=data)
obj_analysis.case_2_candle_length(kline_data_lv3=data)
obj_analysis.case_3_continuous_trend_position(kline_data_lv3=data)
obj_analysis.case_4_process_neg_counts(kline_data_lv3=data, col=7)
obj_analysis.case_5_diff_neg_counts(kline_data_lv3=data, col1=1, col2=4)

In [3]:
status, order_signal_open = asyncio.run(
    obj_order.generate_order_open_signal(
        symbol='XRPUSDT',
        position=1,
        # position=reset_position,
        leverage=5,
        balance=52,
        entry_price=0.5,
        open_timestamp=0,
    )
)

In [4]:
order_signal_open
obj_process.initialize_trading_data(
    symbol='XRPUSDT',
    position=order_signal_open["position"],
    entry_price=order_signal_open["entryPrice"],
)
obj_wallet.add_funds(order_signal=order_signal_open)#, fee_cost)

({'available_funds': 989.105,
  'locked_funds': {'number_of_stocks': 1,
   'profit_and_loss': 0.0,
   'maintenance_margin': 10.83,
   'profit_margin_ratio': 0},
  'total_assets': 999.9350000000001},
 10.83)

In [5]:
pprint(obj_wallet.balance_info)
trade_data.append(order_signal_open)
trade_data.append(obj_wallet.balance_info)

{'available_funds': 989.105,
 'locked_funds': {'maintenance_margin': 10.83,
                  'number_of_stocks': 1,
                  'profit_and_loss': 0.0,
                  'profit_margin_ratio': 0},
 'total_assets': 999.9350000000001}


In [6]:
obj_wallet.get_wallet_status(symbol='XRPUSDT', current_price=1)

{'available_funds': 989.105,
 'locked_funds': {'number_of_stocks': 1,
  'profit_and_loss': 54.15,
  'maintenance_margin': 10.83,
  'profit_margin_ratio': 5.0},
 'total_assets': 1054.085}

In [7]:
obj_order.get_calc_pnl(current_price=0.5, entry_price=0.5, quantity=109, position='long')

0.0

In [8]:
obj_order.generate_order_close_signal(
                symbol='XRPUSDT',
                current_price=0.45,
                wallet_data=obj_wallet.account_balances,
            )

AttributeError: 'OrderManager' object has no attribute 'get_wallet_status'

In [9]:
obj_wallet.remove_funds('XRPUSDT')

({'available_funds': 1054.23065,
  'locked_funds': {'number_of_stocks': 0,
   'profit_and_loss': 0,
   'maintenance_margin': 0,
   'profit_margin_ratio': 0},
  'total_assets': 1054.23065},
 54.35)

In [ ]:
order_signal_open

In [ ]:
print(obj_process.__dict__)
obj_process.remove_trading_data('XRPUSDT')
obj_process.__dict__

In [ ]:
obj_process.__dict__